Dùng để demo quá trình pipeline việc xử lí theo lô

Yêu cầu nghiệp vụ: Các chi nhánh sẽ lưu trữ thông tin vào các file .csv ở local, và các local sẽ đẩy dữ liệu lên server vào mỗi cuối tháng.

In [ ]:
import findspark
findspark.init()

In [ ]:
%pip install delta-spark

In [ ]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkContext

In [ ]:

c = SparkContext.getOrCreate()
spark = SparkSession.builder \
    .appName("Delta on Spark 3.3.x") \
    .master("local[*]") \
    .config("spark.executor.extraClassPath", "/opt/spark/jars/*") \
    .config("spark.driver.extraClassPath", "/opt/spark/jars/*") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

Schema của các bảng:

client_df.printSchema()
root
 |-- client_id: string (nullable = true)
 |-- first_issue_date: string (nullable = true)
 |-- first_redeem_date: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- middle_name: string (nullable = true)
 |-- last_name: string (nullable = true)

product_df.printSchema()
root
 |-- product_id: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- first_issue_date: timestamp (nullable = true)


store_df.printSchema()
root
 |-- store_id: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- store_location: string (nullable = true)
 |-- region: string (nullable = true)
 |-- store_format: string (nullable = true)
 |-- first_issue_date: timestamp (nullable = true)


purchases_df.printSchema()
root
 |-- product_id: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- transaction_id: string (nullable = true)
 |-- transaction_datetime: timestamp (nullable = true)
 |-- regular_points_received: double (nullable = true)
 |-- purchase_sum: double (nullable = true)
 |-- store_id: string (nullable = true)
 |-- product_quantity: double (nullable = true)



Xây dựng các thư viện dữ liệu giả cho user ()

In [ ]:
# naming data dict
first_name = ["Nguyen", "Tran", "Le", "Hoang", "Ngoc", "Phan", "Vu", "Bui", "Do", "Pham", "Vo", "Dang", "Truong", "Ly", "Ha", "Ngo", "Duong", "Nguyen Thi", "Le Thi", 
              "Tran Thi", "Ho Thi", "Nguyen Van", "Le Van", "Tran Van", "Pham Van"]
first_name.append(["John", "Jane", "Michael", "Sarah", "David", "Emily", "Daniel", "Jessica", "Matthew", "Ashley", "Christopher", "Amanda", "Joshua", "Megan", "Andrew",])

middle_name = ["Thi", "Van", "Quoc", "Duc", "Minh", "Thanh", "Hai", "Huu", "Khanh", "Tuan", "Nhat", "Phuoc", "Tam", "Thien", "Trung", "Hanh"]
middle_name.append(["Lee", "Kim", "Park", "Choi", "Lim", "Yoon", "Cho", "Kwon", "Im", "Jung", "Oh", "Seo", "Han", "Ryu", "Hwang"])
middle_name.append(["James", "Robert", "William", "David", "Richard", "Joseph", "Charles", "Thomas", "Christopher", "Daniel", "Matthew", "Anthony", "Mark", "Donald", "Steven"])

last_name = ["Minh", "Thao", "Hanh", "Hoa", "Thuy", "Tam", "Tram", "Trang", "Thuy", "Tuyet", "Mai", "Nhi", "Phuong", "Quynh", "Lan", "Linh", "Thuy", "Yen", "Hanh", "Ngoc", "Thao", "Mai", "Thuy", "Nhi", "Phuong", "Quynh", "Lan", "Linh"]
last_name.append(["Smith", "Johnson", "Williams", "Jones", "Brown", "Davis", "Miller", "Wilson", "Moore", "Taylor", "Anderson", "Thomas", "Jackson", "White", "Harris"])



In [ ]:
gender = ["Male", "Female", "Binary", "Non-binary", "Other"]

In [ ]:
client_ids=[]

In [ ]:
import random, datetime

In [ ]:
def id_random_generator(client_ids):
    """
    Generate a random, 12-digit number for client id
    Make sure no redundance is made by checking the list of client_ids
    """
    client_id = str(random.randint(100000000000, 999999999999))
    return client_id



In [ ]:
def generate_random_user(num_users):
    users = []
    existing_ids = set()

    for i in range(num_users):
        client_id = id_random_generator(existing_ids)

        first_name_random = random.choice(first_name)
        middle_name_random = random.choice(middle_name)
        last_name_random = random.choice(last_name)
        gender_random = random.choice(gender)

        age_random = random.randint(18, 65)

        # Tạo ngày tháng giữa 2020-01-01 đến 2025-06-05
        start_timestamp = int(datetime.datetime(2020, 1, 1).timestamp())
        end_timestamp = int(datetime.datetime(2025, 6, 5).timestamp())

        first_issue_timestamp = random.randint(start_timestamp, end_timestamp)
        first_issue_date = datetime.datetime.fromtimestamp(first_issue_timestamp).strftime('%Y-%m-%d %H:%M:%S')

        first_redeem_timestamp = random.randint(first_issue_timestamp, end_timestamp)
        first_redeem_date = datetime.datetime.fromtimestamp(first_redeem_timestamp).strftime('%Y-%m-%d %H:%M:%S')

        user = {
            'client_id': client_id,
            'first_name': first_name_random,
            'middle_name': middle_name_random,
            'last_name': last_name_random,
            'gender': gender_random,
            'age': age_random,
            'first_issue_date': first_issue_date,
            'first_redeem_date': first_redeem_date
        }

        users.append(user)

    return users

In [ ]:
# run code
num_users = 8386
users = generate_random_user(num_users)
print(users)

Xây dựng thư viện giả dữ liệu cho store

In [ ]:
regions_and_locations = [
    {
        "region": "North East",
        "locations": [
            "Ha Noi", "Hai Phong", "Quang Ninh", "Bac Ninh", "Thai Nguyen", "Hai Duong", "Hung Yen", "Nam Dinh", "Ninh Binh"
        ]
    },
    {
        "region": "Red River Delta",
        "locations": [
            "Ha Nam", "Thai Binh", "Nam Dinh", "Ninh Binh", "Hau Giang", "Vinh Phuc", "Bac Giang", "Bac Ninh"
        ]
    },
    {
        "region": "North West",
        "locations": [
            "Lai Chau", "Dien Bien", "Son La", "Lao Cai", "Yen Bai", "Hoa Binh"
        ]
    },
    {
        "region": "North Central Coast",
        "locations": [
            "Thanh Hoa", "Nghe An", "Ha Tinh", "Quang Binh", "Quang Tri", "Thua Thien Hue"
        ]
    },
    {
        "region": "South Central Coast",
        "locations": [
            "Da Nang", "Quang Nam", "Quang Ngai", "Binh Dinh", "Phu Yen", "Khanh Hoa"
        ]
    },
    {
        "region": "Central Highlands",
        "locations": [
            "Dak Lak", "Dak Nong", "Lam Dong", "Gia Lai", "Kon Tum"
        ]
    },
    {
        "region": "South East",
        "locations": [
            "Ho Chi Minh City (HCMC)", "Ba Ria-Vung Tau", "Binh Duong", "Dong Nai", "Tay Ninh", "Binh Phuoc", "Long An", "Tay Ninh"
        ]
    },
    {
        "region": "Mekong River Delta",
        "locations": [
            "Can Tho", "An Giang", "Kien Giang", "Dong Thap", "Vinh Long", "Tra Vinh", "Soc Trang", "Bac Lieu", "Ca Mau"
        ]
    }
]

In [ ]:
store_formats = ["Store", "Shop", "Supermarket", "Convenience Store", "Online Store", "Retailer", "Wholesaler"]

In [ ]:
def get_data_from_locations():
    """
    Return a combination of the following attributes:
    - region: randomly from regions_and_locations
    - location: randomly from locations in the region
    - store_name: randomly from locations with a random 3-digit number
    """
    region = random.choice(regions_and_locations)
    location = random.choice(region["locations"])
    store_name = f"{location} {random.randint(100, 999)}"

    return {
        "region": region["region"],
        "location": location,
        "store_name": store_name,
    }



store_df.printSchema()
root
 |-- store_id: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- store_location: string (nullable = true)
 |-- region: string (nullable = true)
 |-- store_format: string (nullable = true)
 |-- first_issue_date: timestamp (nullable = true)

In [ ]:
def generate_random_store(num_stores):
    """
    Generate a random store with schema from stores_df
    id: random 12-digit number
    store_name, store_location, region: randomly from get_data_from_locations()
    store_format: randomly from store_formats
    first_issue_date: random date between 2020-01-01 and 2025-06-05
    """

    stores = []
    existing_ids = set()

    for i in range(num_stores):
        store_id = id_random_generator(existing_ids)

        store_data = get_data_from_locations()
        store_format_random = random.choice(store_formats)

        # Tạo ngày tháng giữa 2020-01-01 đến 2025-06-05
        start_timestamp = int(datetime.datetime(2020, 1, 1).timestamp())
        end_timestamp = int(datetime.datetime(2025, 6, 5).timestamp())

        first_issue_timestamp = random.randint(start_timestamp, end_timestamp)
        first_issue_date = datetime.datetime.fromtimestamp(first_issue_timestamp).strftime('%Y-%m-%d %H:%M:%S')

        store = {
            'store_id': store_id,
            'store_name': store_data["store_name"],
            'store_location': store_data["location"],
            'region': store_data["region"],
            'store_format': store_format_random,
            'first_issue_date': first_issue_date
        }

        stores.append(store)

    return stores

In [ ]:
# run code for stores
num_stores = 100
stores = generate_random_store(num_stores)
print(stores)

product_df.printSchema()
root
 |-- product_id: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- first_issue_date: timestamp (nullable = true)

Tạo thư viện cho product
product_df.printSchema()
root
 |-- product_id: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- first_issue_date: timestamp (nullable = true)


In [ ]:
brands_and_products = [
    {"brand": "Apple", "products": ["iPhone", "iPad", "MacBook", "Apple Watch", "AirPods"]},
    {"brand": "Samsung", "products": ["Galaxy S", "Galaxy Note", "Galaxy A", "Galaxy Tab", "Galaxy Watch"]},
    {"brand": "Sony", "products": ["PlayStation", "Xperia", "Bravia", "Alpha Camera", "Walkman"]},
    {"brand": "LG", "products": ["LG G", "LG V", "LG Q", "LG K", "LG Stylo"]},
    {"brand": "Nokia", "products": ["Nokia 1", "Nokia 2", "Nokia 3", "Nokia 5", "Nokia 7"]},
    {"brand": "Huawei", "products": ["P Series", "Mate Series", "Nova Series", "Y Series", "Honor"]},
    {"brand": "Xiaomi", "products": ["Mi Series", "Redmi Series", "Poco Series", "Black Shark"]},
    {"brand": "Oppo", "products": ["Reno Series", "Find Series", "A Series"]},
    {"brand": "Vivo", "products": ["Vivo V Series", "Vivo X Series", "Vivo Y Series"]},
    {"brand": "Motorola", "products": ["Moto G", "Moto E", "Moto Z"]},
    {"brand": "OnePlus", "products": ["OnePlus 9", "OnePlus 8", "OnePlus Nord"]},
    {"brand": "Google", "products": ["Pixel", "Nest", "Chromecast"]},
    {"brand": "Dell", "products": ["XPS", "Inspiron", "Alienware"]},
    {"brand": "HP", "products": ["Pavilion", "Envy", "Spectre"]},
    {"brand": "Lenovo", "products": ["ThinkPad", "IdeaPad", "Legion"]},
    {"brand": "Asus", "products": ["ZenBook", "VivoBook", "ROG"]},
    {"brand": "Acer", "products": ["Aspire", "Predator", "Swift"]},
    {"brand": "Microsoft", "products": ["Surface Pro", "Surface Laptop", "Surface Book"]},
    {"brand": "Razer", "products": ["Razer Blade", "Razer Phone"]},
    {"brand": "Logitech", "products": ["Logitech G", "Logitech MX"]},
    {"brand": "Corsair", "products": ["Corsair K70", "Corsair M65"]},
    {"brand": "SteelSeries", "products": ["SteelSeries Rival", "SteelSeries Apex"]},
    {"brand": "HyperX", "products": ["HyperX Cloud", "HyperX Alloy"]},
    {"brand": "TP-Link", "products": ["Archer", "Deco", "Kasa"]},
    {"brand": "Netgear", "products": ["Nighthawk", "Orbi"]},
    {"brand": "Linksys", "products": ["Velop", "WRT"]},
    {"brand": "D-Link", "products": ["DIR Series", "DAP Series"]},
    {"brand": "ZTE", "products": ["Axon", "Blade"]},
    {"brand": "TCL", "products": ["Alcatel", "BlackBerry"]},
    {"brand": "Realme", "products": ["Realme 8", "Realme Narzo"]},
    {"brand": "Infinix", "products": ["Infinix Zero", "Infinix Hot"]},
    {"brand": "Tecno", "products": ["Tecno Camon", "Tecno Spark"]},
    {"brand": "Vivo", "products": ["Vivo V21", "Vivo Y20"]},
    {"brand": "Honor", "products": ["Honor 50", "Honor V40"]},
    {"brand": "ZTE", "products": ["ZTE Axon", "ZTE Blade"]},
    {"brand": "Alcatel", "products": ["Alcatel 1", "Alcatel 3"]},
    {"brand": "BlackBerry", "products": ["BlackBerry Key2", "BlackBerry Motion"]},
    {"brand": "Panasonic", "products": ["Panasonic Lumix", "Panasonic Toughbook"]},
    {"brand": "Fujitsu", "products": ["Fujitsu Lifebook", "Fujitsu Stylistic"]},
    {"brand": "Toshiba", "products": ["Toshiba Satellite", "Toshiba Tecra"]},
    {"brand": "Sharp", "products": ["Sharp Aquos", "Sharp Plasmacluster"]},
    {"brand": "BenQ", "products": ["BenQ SW Series", "BenQ PD Series"]},
    {"brand": "Epson", "products": ["Epson EcoTank", "Epson WorkForce"]},
    {"brand": "Canon", "products": ["Canon EOS", "Canon PIXMA"]},
    {"brand": "Nikon", "products": ["Nikon D Series", "Nikon Z Series"]},
    {"brand": "Fujifilm", "products": ["Fujifilm X Series", "Fujifilm GFX"]},
    {"brand": "Olympus", "products": ["Olympus OM-D", "Olympus Tough"]},
    {"brand": "GoPro", "products": ["GoPro HERO", "GoPro MAX"]},
    {"brand": "DJI", "products": ["DJI Mavic", "DJI Phantom"]},
    {"brand": "Parrot", "products": ["Parrot Anafi", "Parrot Bebop" ]},
    {"brand": "Yuneec", "products": ["Yuneec Typhoon", "Yuneec Breeze"]},
    {"brand": "Autel Robotics", "products": ["Autel EVO", "Autel X-Star"]},
    {"brand": "Skydio", "products": ["Skydio 2", "Skydio X2"]},
    {"brand": "Hubsan", "products": ["Hubsan Zino", "Hubsan H501S"]},
    {"brand": "Holy Stone", "products": ["Holy Stone HS720", "Holy Stone HS110D"]},
    {"brand": "Potensic", "products": ["Potensic D80", "Potensic D85"]},
    {"brand": "Snaptain", "products": ["Snaptain SP350", "Snaptain SP3000"]},
    {"brand": "Ruko", "products": ["Ruko F11", "Ruko F11 Pro"]},
    {"brand": "Contixo", "products": ["Contixo F22", "Contixo F18"]},
    {"brand": "DBPOWER", "products": ["DBPOWER MJX X400W", "DBPOWER UDI U842"]},
    {"brand": "Syma", "products": ["Syma X5C", "Syma X8G"]},
    {"brand": "JJRC", "products": ["JJRC H68", "JJRC H37"]},
]

In [ ]:
def generate_random_products(product_num):
    """
    Generate a random product with schema from products_df
    id: random 12-digit number
    brand: randomly from brands_and_products
    product: randomly from products in the brand
    first_issue_date: random date between 2020-01-01 and 2025-06-05
    price: random between 1000 and 100000
    dont concerns about product id being duplicated
    level_1 -> 4: random 12-digit number
    """

    products = []
    existing_ids = set()

    for i in range(product_num):
        product_id = id_random_generator(existing_ids)

        brand_data = random.choice(brands_and_products)
        product_name_random = random.choice(brand_data["products"])
        brand_name_random = brand_data["brand"]

        # Tạo ngày tháng giữa 2020-01-01 đến 2025-06-05
        start_timestamp = int(datetime.datetime(2020, 1, 1).timestamp())
        end_timestamp = int(datetime.datetime(2025, 6, 5).timestamp())

        first_issue_timestamp = random.randint(start_timestamp, end_timestamp)
        first_issue_date = datetime.datetime.fromtimestamp(first_issue_timestamp).strftime('%Y-%m-%d %H:%M:%S')

        price_random = random.randint(1000, 100000)

        product = {
            'product_id': product_id,
            'product_name': product_name_random,
            'brand_name': brand_name_random,
            'first_issue_date': first_issue_date,
            'price': price_random
        }

        products.append(product)

    return products




In [ ]:
product_num = 1000
products = generate_random_products(product_num)
print(products)

root
 |-- product_id: string (nullable = true)
 |-- client_id: string (nullable = true)
 |-- transaction_id: string (nullable = true)
 |-- transaction_datetime: timestamp (nullable = true)
 |-- regular_points_received: double (nullable = true)
 |-- purchase_sum: double (nullable = true)
 |-- store_id: string (nullable = true)
 |-- product_quantity: double (nullable = true)


In [ ]:
def create_transaction(transaction_number):
    """
    Generate a random transaction with schema from transactions_df
    id: random 12-digit number
    client_id: randomly from users
    store_id: randomly from stores
    product_id: randomly from products
    transaction_date: random date between 2020-01-01 and 2025-06-05
    amount: random between 1 and 10
    purchase_sum: amount * price
    dont concerns about transaction id being duplicated
    """

    transactions = []
    existing_ids = set()

    for i in range(transaction_number):
        transaction_id = id_random_generator(existing_ids)

        client_id_random = random.choice(users)["client_id"]
        store_id_random = random.choice(stores)["store_id"]
        product_id_random = random.choice(products)["product_id"]

        # Tạo ngày tháng giữa 2020-01-01 đến 2025-06-05
        start_timestamp = int(datetime.datetime(2020, 1, 1).timestamp())
        end_timestamp = int(datetime.datetime(2025, 6, 5).timestamp())

        transaction_timestamp = random.randint(start_timestamp, end_timestamp)
        transaction_date = datetime.datetime.fromtimestamp(transaction_timestamp).strftime('%Y-%m-%d %H:%M:%S')

        amount_random = random.randint(1, 10)

        transaction = {
            'transaction_id': transaction_id,
            'client_id': client_id_random,
            'store_id': store_id_random,
            'product_id': product_id_random,
            'transaction_date': transaction_date,
            'amount': amount_random
        }

        transactions.append(transaction)

    return transactions

In [ ]:
transaction_number = 10000
transactions = create_transaction(transaction_number)

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType

# Schema khách hàng (clients/users)
schema_client = StructType([
    StructField("client_id", StringType(), True),
    StructField("first_issue_date", StringType(), True),
    StructField("first_redeem_date", StringType(), True),
    StructField("age", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("middle_name", StringType(), True),
    StructField("last_name", StringType(), True),
])

# Schema sản phẩm
schema_products = StructType([
    StructField("product_id", StringType(), True),
    StructField("level_1", StringType(), True),
    StructField("level_2", StringType(), True),
    StructField("level_3", StringType(), True),
    StructField("level_4", StringType(), True),
    StructField("brand_name", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("product_price", DoubleType(), True),
    StructField("first_issue_date", TimestampType(), True),
])

# Schema cửa hàng
schema_stores = StructType([
    StructField("store_id", StringType(), True),
    StructField("store_name", StringType(), True),
    StructField("store_location", StringType(), True),
    StructField("region", StringType(), True),
    StructField("store_format", StringType(), True),
    StructField("first_issue_date", TimestampType(), True),
])

# Schema purchase
schema_purchases = StructType([
    StructField("product_id", StringType(), True),
    StructField("client_id", StringType(), True),
    StructField("transaction_id", StringType(), True),
    StructField("transaction_datetime", TimestampType(), True),
    StructField("regular_points_received", DoubleType(), True),
    StructField("purchase_sum", DoubleType(), True),
    StructField("store_id", StringType(), True),
    StructField("product_quantity", DoubleType(), True),
])

In [ ]:
%pip install python-dateutil

In [ ]:
def create_dataframe_with_schema(data_list, schema):
    """
    Tạo DataFrame Spark trực tiếp từ list dict đã xử lý đúng kiểu, dựa trên schema cung cấp.
    """
    from pyspark.sql import SparkSession, Row
    from pyspark.sql.types import StringType, DateType, TimestampType, ArrayType
    from datetime import datetime
    import dateutil.parser

    spark = SparkSession.builder.getOrCreate()

    processed_data = []

    for item in data_list:
        processed_item = {}
        for field in schema.fields:
            name = field.name
            value = item.get(name, None)

            if isinstance(field.dataType, (DateType, TimestampType)) and value is not None:
                # Chuyển chuỗi thành date/datetime
                if isinstance(value, str):
                    try:
                        dt = dateutil.parser.parse(value)
                        if isinstance(field.dataType, DateType):
                            processed_item[name] = dt.date()
                        elif isinstance(field.dataType, TimestampType):
                            processed_item[name] = dt
                    except:
                        processed_item[name] = None
                elif isinstance(value, datetime):
                    if isinstance(field.dataType, DateType):
                        processed_item[name] = value.date()
                    elif isinstance(field.dataType, TimestampType):
                        processed_item[name] = value
                else:
                    processed_item[name] = value

            elif isinstance(field.dataType, StringType):
                if value is not None:
                    # Chuyển list hoặc dict thành String nếu cần
                    if isinstance(value, (list, dict)):
                        import json
                        processed_item[name] = json.dumps(value)
                    else:
                        processed_item[name] = str(value)

            elif isinstance(field.dataType, ArrayType):
                # Nếu không phải list, cố gắng parse hoặc đóng gói
                if not isinstance(value, list):
                    if isinstance(value, str):
                        try:
                            import json
                            processed_item[name] = json.loads(value)
                        except:
                            processed_item[name] = [value]
                    else:
                        processed_item[name] = [value]
                else:
                    processed_item[name] = value

            else:
                # các kiểu còn lại
                processed_item[name] = value

        # nếu có key nào không có trong dict gốc, set là None
        for field in schema.fields:
            if field.name not in processed_item:
                processed_item[field.name] = None

        processed_data.append(processed_item)

    # Chuyển sang Spark DataFrame
    df = spark.createDataFrame(processed_data, schema=schema)
    return df

In [ ]:
users_df = create_dataframe_with_schema(users, schema_client)
products_df = create_dataframe_with_schema(products, schema_products)
stores_df = create_dataframe_with_schema(stores, schema_stores)
transactions_df = create_dataframe_with_schema(transactions, schema_purchases)

In [ ]:
# write df to csv.One file for each df
users_df.coalesce(1).write.csv("users", header=True, mode="overwrite")
products_df.coalesce(1).write.csv("products", header=True, mode="overwrite")
stores_df.coalesce(1).write.csv("stores", header=True, mode="overwrite")
transactions_df.coalesce(1).write.csv("transactions", header=True, mode="overwrite")

In [ ]:
!scp ./products.csv macminhduy2004@34.70.6.241:/app/airflow/docker/ELT-prj/files
!yes